
# A rush incursion in SAT and SMT (with z3)



# 1- SAT -- Boolean satisfiability solving
## quantified free propositional formulas
Alexandre Madeira




## In this class we will use z3 Theorem Prover
"Z3 is a state-of-the art theorem prover from Microsoft Research. It can be used to check the satisfiability of logical formulas over one or more theories. Z3 offers a compelling match for software analysis and verification tools, since several common software constructs map directly into supported theories."


In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 4.9 MB/s eta 0:00:0000:0100:01


In [2]:
from z3 import *

Let us see how to use z3 to check the satisfability or propositional formulas

Introduce the set of propositions:

In [3]:
q = Bool("q")
p = Bool("p")

Define a formula:

In [4]:
formula = Or(And(p,q),Or(Not(p),q))
formula

Or(And(p, q), Or(Not(p), q))

Setup a z3 model 

In [5]:
s = Solver()
s.add(formula)

Check if the formula is sat:

In [6]:
s.check()

sat

Show the interpretation:

In [7]:
print(s.model())

[q = True, p = True]


Check if the formula is a tautology:

In [8]:
prove(formula)

counterexample
[p = True, q = False]


Now an example of a tautology/theorem: 


In [9]:
formula2 = Or(p,Not(p))
formula2

Or(p, Not(p))

In [10]:
prove(formula2)

proved


## Exercise 1

Solve the following puzzle:


You want to throw a party, respecting people’s incompatibilities.
You know that:
- John comes if Mary or Ann comes. 
- Ann Comes if Mary does not come. 
- If Ann comes, John does not.

Can you invite people under these constraints?

(Logic in Action. van Benthem et al. 2016)

In [19]:

# Create a solver
s = Solver()

# Create some propositional variables
john = Bool('john') # John comes
mary = Bool('mary') # Mary comes
ann = Bool('ann') # Ann comes


# Add constraints
#
s.add(
    Implies(Or(mary, ann), john),
    Implies(Not(mary), ann),
    Implies(ann, Not(john)),
)

print(s.check())
# Check satisfiability
if s.check() == sat:
    # Print model
    m = s.model()
    print(f"john = {m[john]}, mary = {m[mary]}, ann = {m[ann]}")

sat
john = True, mary = True, ann = False


## Exercise 2

Suppose that liars always speak what is false, and truth-tellers always speak what is true. Further suppose that Amy, Bob, and Cal are each either a liar or truth-teller. Amy says, “Bob is a liar.” Bob says, “Cal is a liar.” Cal says, “Amy and Bob are liars.” Which, if any, of these people are truth-tellers?

(Neller, Markov, Russel. Clue Deduction: Professor Plum Teaches Logic (2016))

In [25]:
s = Solver()

# Create some propositional variables
a = Bool('a') # Ann is a truth-teller
b = Bool('b') # Bob is a truth-teller
c = Bool('c') # Cal is a truth-teller


s.add(
    # Add constraints
    # Amy says, “Bob is a liar.”
    Implies(Not(b), a),
    Implies(a, Not(b)),

    #Bob says, “Cal is a liar.”
    Implies(Not(c), b),
    Implies(b, Not(c)),

    # Cal says, “Amy and Bob are liars.”
    Implies(And(Not(a), Not(b)), c),
    Implies(c, And(Not(a), Not(b))),
)

# Check satisfiability
if s.check() == sat:
    # Print model
    m = s.model()
    print(f"a = {m[a]}, b = {m[b]}, c = {m[c]}")
else: print('there are no solutions' )    

a = False, b = True, c = False


There are another possible solution?

In [26]:
s.add ( Not(b) )

In [27]:

# Check satisfiability
if s.check() == sat:
    # Print model
    m = s.model()
    print(f"a = {m[a]}, b = {m[b]}, c = {m[c]}")
else: print('there are no solutions' )    

there are no solutions


# 2- SMT -- satisfiability modulo theories 
## quantified free formulas with other theories


We can now deal with a number of theories, including with numerical domains. 

## Example -- playing with integers
Define integer variables and setup a model:

In [28]:
x = Int('x')
y = Int('y')
s = Solver()

Then we include our constraints (e.g. check if there is an integer solution to $x+y=3$)

In [29]:
s.add(x+y == 3)
if s.check() == sat:
    m = s.model()
    print(s.model())
else: print ('unsat')    

[y = 0, x = 3]


## Exercise: prove that the solution is not unique

In [30]:
s.add(x!=3)

In [31]:

#check satisfaction and print the model
if s.check() == sat:
    m = s.model()
    print(s.model())
else: print ('unsat')

[x = 0, y = 3]


# Example -- playing with reals
Define real variables and setup a model:


In [32]:
x = Real('x')
y = Real('y')

s = Solver()

then we can use z3 to solve equations. For instance, to solve $x^2 +2x =1$: 

In [33]:
s.add(x**2 + 2 * x == 1 )

In [34]:
if s.check() == sat:
    m = s.model()
    print(s.model())


[x = -2.4142135623?]


## Exercise: Calculate the roots of $x^3+3x^2+4x+2=0$ and prove that the root is unique

Firstly, let us solve the equation

In [41]:
x = Real('x')
s = Solver()
s.add(x**3 + 3*x**2 + 4*x + 2 == 0)



In [42]:
if s.check() == sat:
    m = s.model()
    print(s.model())
else: 
    print("Unsat")

[x = -1]


Then, let us show that it is unique:

In [43]:
s.add(x!=-1)

In [44]:
s.check()

unsat

# Playing with z3

# 8 Queens puzzle
The 8 queens problem is a classic problem in computer science where the goal is to place 8 queens on a chessboard in such a way that no two queens can attack each other.

## Implementation 1:

In [45]:
from z3 import *


We know each queen must be in a different row.
 So, we represent each queen by a single integer: the column position

In [46]:
Q = [ Int('Q_%i' % (i + 1)) for i in range(8) ]
print(Q)

[Q_1, Q_2, Q_3, Q_4, Q_5, Q_6, Q_7, Q_8]


Each queen is in a column {1, ... 8 }

In [47]:
val_c = [ And(1 <= Q[i], Q[i] <= 8) for i in range(8) ]


At most one queen per column

In [49]:
col_c = [ Distinct(Q) ]


Diagonal constraint:


In [50]:
diag_c = [ If(i == j,
              True,
              And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i))
           for i in range(8) for j in range(i) ]


A solution for the puzzle:

In [51]:
solve(val_c + col_c + diag_c)

[Q_3 = 8,
 Q_1 = 4,
 Q_7 = 5,
 Q_8 = 7,
 Q_5 = 1,
 Q_4 = 6,
 Q_2 = 2,
 Q_6 = 3]


## Implementation 2

In [52]:
from z3 import *

We consider a variable for each position

- $x[i][j]==1$ if``there is a queen at position $(i,j)$
- $x[i][j]==0$ if``there is not a queen at position $(i,j)$


In [53]:
X = [[Int("x_%s_%s" % (i+1, j+1)) for j in range(8)] for i in range(8) ]

cells_c = [Or(X[i][j] == 0, X[i][j] == 1) for i in range(8) for j in range(8) ]


Constraint 1: there is exactly one queen per row

In [54]:
rows_c = Sum(X[i]) == 1 for i in range(8)]

Constraint 2: there is exactly one queen per column

In [55]:
cols_c = [Sum([X[i][j] for i in range(8)]) == 1 for j in range(8) ]

Constraint 3: there is exactly one queen per diagonal

In [56]:
diagonals_c = [Implies(And(X[i][j] == 1, X[k][h] == 1,
            i != k, j != h), abs(k - i) != abs(j - h))
            for i in range(8) for j in range(8) 
            for k in range(8) for h in range(8)]


Setup the model and check:

In [57]:
eight_queens_c = cells_c + rows_c + cols_c + diagonals_c

s = Solver()
s.add(eight_queens_c)
if s.check() == sat:
    m = s.model()
    r = [[m.evaluate(X[i][j]) for j in range(8)] for i in range(8)]
    print_matrix(r)
else:
    print ("failed to solve")

[[0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0]]


## Some interesting links about z3



Programming Z3 - https://theory.stanford.edu/~nikolaj/programmingz3.html
Nikolaj Bjorner's tutorial https://youtu.be/nGwyNmsxX6I
Hakank's examples http://www.hakank.org/z3/
Yurichev's book "SMT by Example" https://yurichev.com/writings/SAT_SMT_by_example.pdf
